### BES Location into genome through the rerult of blast
```bash
blastn -db Panicum_miliaceum_longmi4.fna -query PM-B-BES.fasta -outfmt '6 qaccver saccver pident length mismatch gapopen qlen qstart qend slen sstart send evalue bitscore sstrand qcovhsp' -qcov_hsp_perc 99 -perc_identity 99 -lcase_masking > PM-B-BES_longmi4.unmasked.blast
```

#### bedtogff3
```python
import pandas
dff = pandas.read_table('PM-B-BES.bed',header=None,names=['chr','start','end','id','Q','strand'])

gff = pandas.DataFrame(columns=['chr','source','type','start','end','score','strand','phase','attributes'])

for chr in set(dff['chr']):
    df = dff.loc[dff['chr']==chr,]
    for i in set([i[:-2] for i in df.id]):
        if i+'_R' in list(df.id) and i+'_F' in list(df.id):
            BES_R = df.loc[df['id'] == str(i + '_R'),:]
            BES_F = df.loc[df['id'] == str(i + '_F'),:]
            start = min(BES_F.iat[0,1],BES_F.iat[0,2],BES_R.iat[0,1],BES_R.iat[0,2])
            end = max(BES_F.iat[0,1],BES_F.iat[0,2],BES_R.iat[0,1],BES_R.iat[0,2])
            if abs(start - end) < 500000:
                gff.loc[i] = [BES_R.iat[0,0],'GResource','BAC',str(start),str(end),'.','.','.',str('ID='+i+';Name='+i)]
                gff.loc[str(i+'-F')] = [BES_F.iat[0,0],'GResource','BAC_end',BES_F.iat[0,1],BES_F.iat[0,2],BES_F.iat[0,4],BES_F.iat[0,5],'.',str('Parent='+i)]
                gff.loc[str(i+'-R')] = [BES_R.iat[0,0],'GResource','BAC_end',BES_R.iat[0,1],BES_R.iat[0,2],BES_R.iat[0,4],BES_R.iat[0,5],'.',str('Parent='+i)]

gff.to_csv('PM-B-BES.gff3',index=None,sep='\t')
```

In [1]:
%%time
import pandas as pd
import collections

blast = pd.read_table('PM-B-BES_longmi4.unmasked.blast',header=None)[:10]

CPU times: user 955 ms, sys: 267 ms, total: 1.22 s
Wall time: 1.91 s


In [31]:
for i in range(blast.shape[0]):    # add BAC id to the last column
    blast.loc[i,15] = blast.loc[i,0].split('_')[0]
blast[:3]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,25A01_F,6,100.0,379,0,0,379,1,379,52839179,4968299,4968677,0.000000e+00,701.0,+,25A01
1,25A02_F,18,100.0,455,0,0,455,1,455,32237550,15834864,15834410,0.000000e+00,841.0,-,25A02
2,25A03_F,10,100.0,243,0,0,243,1,243,44648547,30469114,30468872,6.960000e-125,449.0,-,25A03


In [32]:
freq = collections.Counter(blast[0])   ## extract single-hit BES id.
single_hit_BES_id = [i for i in freq if freq[i] == 1]

BAC_id = set([i.split('_')[0] for i in single_hit_BES_id])

gff = pd.DataFrame(columns=['chr','source','type','start','end','score','strand','phase','attributes'])

for i in BAC_id:
    BAC = blast[blast[15] == i]
    if BAC.shape[0] == 2 and i + '_F' in list(BAC[0]) and i + '_R' in list(BAC[0]):
        BES_R = BAC.loc[BAC[0] == str(i + '_R'),:]
        BES_F = BAC.loc[BAC[0] == str(i + '_F'),:]
        start = min(BES_F.iat[0,10],BES_F.iat[0,11],BES_R.iat[0,10],BES_R.iat[0,11])
        end = max(BES_F.iat[0,10],BES_F.iat[0,11],BES_R.iat[0,10],BES_R.iat[0,11])
        if abs(start - end) < 250000:
            gff.loc[i] = [BES_R.iat[0,1],'GResource','BAC', \
                          str(start),str(end),'.','.','.',str('ID='+i+';Name='+i)]
            gff.loc[str(i+'-F')] = [BES_F.iat[0,1], 'GResource','BAC_end', \
                                    min(BES_F.iat[0,10],BES_F.iat[0,11]), max(BES_F.iat[0,10],BES_F.iat[0,11]), \
                                    '.', BES_F.iat[0,14], '.', str('Parent='+i)]
            gff.loc[str(i+'-R')] = [BES_R.iat[0,1],'GResource','BAC_end', \
                                    min(BES_R.iat[0,10],BES_R.iat[0,11]),max(BES_R.iat[0,10],BES_R.iat[0,11]), \
                                    '.', BES_R.iat[0,14], '.', str('Parent='+i)]
    elif BAC.shape[0] == 1:
        gff.loc[i] = [BAC.iat[0,1], 'GResource','BAC_end', \
                                    min(BAC.iat[0,10],BAC.iat[0,11]), max(BAC.iat[0,10],BAC.iat[0,11]), \
                                    '.', BAC.iat[0,14], '.', str('ID='+i+';Name='+i)]

In [33]:
gff.to_csv('PM-B-BES.gff3',index=None,sep='\t')

In [34]:
blast

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,25A01_F,6,100.000,379,0,0,379,1,379,52839179,4968299,4968677,0.000000e+00,701.0,+,25A01
1,25A02_F,18,100.000,455,0,0,455,1,455,32237550,15834864,15834410,0.000000e+00,841.0,-,25A02
2,25A03_F,10,100.000,243,0,0,243,1,243,44648547,30469114,30468872,6.960000e-125,449.0,-,25A03
3,25A04_F,18,100.000,367,0,0,367,1,367,32237550,17925736,17925370,0.000000e+00,678.0,-,25A04
4,25A04_F,18,100.000,367,0,0,367,1,367,32237550,18471516,18471150,0.000000e+00,678.0,-,25A04
5,25A04_F,18,100.000,367,0,0,367,1,367,32237550,18770300,18769934,0.000000e+00,678.0,-,25A04
6,25A04_F,18,100.000,367,0,0,367,1,367,32237550,18820266,18819900,0.000000e+00,678.0,-,25A04
7,25A04_F,18,100.000,367,0,0,367,1,367,32237550,28667986,28667620,0.000000e+00,678.0,-,25A04
8,25A04_F,18,99.728,367,1,0,367,1,367,32237550,11146687,11146321,0.000000e+00,673.0,-,25A04
9,25A04_F,18,99.728,367,1,0,367,1,367,32237550,17133110,17132744,0.000000e+00,673.0,-,25A04


## 3. Annotation BAC location by both short and long BES



* merge two blast result from short and long BES into a file
* s/plus/+/, s/minus/-/
* 
* 
* 
* 

In [1]:
import pandas as pd
import collections

blast = pd.read_table('../workspace/short_and_long.blast',header=None)

#add two field "name" and "end" into 17th and 18th respectively
for i in range(blast.shape[0]): 
    blast.loc[i, 16] = blast.loc[i, 0].split('_')[0]
    blast.loc[i, 17] = blast.loc[i, 0].split('_')[-1]
blast[:3]

# create a new DataFrame with 9 columns of gff3 format
gff = pd.DataFrame(columns=['chr','source','type','start','end','score','strand','phase','attributes'])

# group by BAC name
for BES_name, BES_groups in blast.groupby(16):
    print(BES_name)
    BES_F = None
    BES_R = None
    
    # group by BAC end
    for strand,BES_group in BES_groups.groupby([17]):
        BES_group = BES_group.sort_values(by = [13], ascending = False)[:1] # get the maximum bitscore of record
        if BES_group.iloc[0,17] == 'F':
            BES_F = BES_group
        elif BES_group.iloc[0,17] == 'R':
            BES_R = BES_group
            
    if BES_F is not None and BES_R is not None:  # Both of ends are record
        start = min(BES_F.iat[0,10],BES_F.iat[0,11],BES_R.iat[0,10],BES_R.iat[0,11])
        end = max(BES_F.iat[0,10],BES_F.iat[0,11],BES_R.iat[0,10],BES_R.iat[0,11])
        if abs(start - end) < 250000:    # filt by the maximum of length of BAC inserted fragment
            gff.loc[BES_name] = [BES_R.iat[0,1],'GResource','BES', \
                          str(start),str(end),'.','.','.',str('ID='+BES_name+';Name='+BES_name)]
            gff.loc[str(BES_name+'-F')] = [BES_F.iat[0,1], 'GResource','BES_end', \
                                    min(BES_F.iat[0,10],BES_F.iat[0,11]), max(BES_F.iat[0,10],BES_F.iat[0,11]), \
                                    '.', BES_F.iat[0,14], '.', str('Parent='+BES_name)]
            gff.loc[str(BES_name+'-R')] = [BES_R.iat[0,1],'GResource','BES_end', \
                                    min(BES_R.iat[0,10],BES_R.iat[0,11]),max(BES_R.iat[0,10],BES_R.iat[0,11]), \
                                    '.', BES_R.iat[0,14], '.', str('Parent='+BES_name)]
    elif BES_F is None:   # Only reverse BES is recorded
        gff.loc[BES_name] = [BES_R.iat[0,1], 'GResource','BES_end', \
                                    min(BES_R.iat[0,10],BES_R.iat[0,11]), max(BES_R.iat[0,10],BES_R.iat[0,11]), \
                                    '.', BES_R.iat[0,14], '.', str('ID='+BES_name+';Name='+BES_name)]
    elif BES_R is None:   # Only forward is recorded
        gff.loc[BES_name] = [BES_F.iat[0,1], 'GResource','BES_end', \
                                    min(BES_F.iat[0,10],BES_F.iat[0,11]), max(BES_F.iat[0,10],BES_F.iat[0,11]), \
                                    '.', BES_F.iat[0,14], '.', str('ID='+BES_name+';Name='+BES_name)]



25A01
25A02
25A03
25A04
25A05
25A06
25A07
25A08
25A09
25A10
25A11
25A12
25A13
25A14
25A15
25A16
25A17
25A18
25A19
25A20
25A21
25A22
25A23
25A24


In [ ]:
gff.to_csv('PM-B-BES.gff3',index=None,sep='\t')

In [2]:
gff

,chr,source,type,start,end,score,strand,phase,attributes
25A01,6,GResource,BES_end,4968299,5008417,.,+,.,ID=25A01;Name=25A01
25A02,18,GResource,BES_end,15834410,15834864,.,-,.,ID=25A02;Name=25A02
25A03,10,GResource,BES,30369637,30469114,.,.,.,ID=25A03;Name=25A03
25A03-F,10,GResource,BES_end,30468872,30469114,.,-,.,Parent=25A03
25A03-R,10,GResource,BES_end,30369637,30369930,.,+,.,Parent=25A03
25A05,2,GResource,BES_end,15041792,15066280,.,+,.,ID=25A05;Name=25A05
25A07,1,GResource,BES,63813657,63903632,.,.,.,ID=25A07;Name=25A07
25A07-F,1,GResource,BES_end,63813657,63903632,.,+,.,Parent=25A07
25A07-R,1,GResource,BES_end,63813657,63903631,.,-,.,Parent=25A07
25A08,8,GResource,BES_end,24289290,24289769,.,-,.,ID=25A08;Name=25A08


In [4]:
a = [j for i,j in blast.groupby(16)]

In [15]:
[j.sort_values(by=[13],ascending=False) for i,j in a[14].groupby(17)]

[               0   1        2      3   4   5      6   7      8         9   \
 420  25A15_ctg1_F  10   99.933  35863   4   3  75060   1  35843  44648547   
 386     25A15_2_F   9  100.000    224   0   0    224   1    224  45112342   
 385     25A15_1_F  10  100.000    194   0   0    194   1    194  44648547   
 
            10        11             12       13 14   15     16 17  
 420  41631708  41667570   0.000000e+00  66076.0  +   48  25A15  F  
 386  28407190  28406967  2.320000e-114    414.0  -  100  25A15  F  
 385  41631708  41631901   9.330000e-98    359.0  +  100  25A15  F  ,
                0   1        2      3   4   5      6   7      8         9   \
 437  25A15_ctg1_R  16  100.000  30281   0   0  30285   1  30281  33613985   
 438  25A15_ctg2_R  10   99.973  25868   3   2  25866   1  25866  44648547   
 405       25A15_R  10  100.000    433   0   0    433   1    433  44648547   
 
            10        11   12       13 14   15     16 17  
 437  29325736  29295456  0.0  55919

### a BAC record
* split forward and reverse
* merge the same loci
* combination

In [1]:
class BAC_Recorder:
    def __init__(self, record):
        self._record = record
        self._name = self._record.iloc[0, 16]
        self._Forward = pd.DataFrame(columns=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])
        self._Reverse = pd.DataFrame(columns=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])
        self._gff = pd.DataFrame(columns=['chr','source','type','start','end','score','strand','phase','attributes'])
        
        
        self.split()
        self._Forward = self.merge(BES=self._Forward)
        self._Reverse = self.merge(BES=self._Reverse)
        self.conbinates()
        
    def split(self):
        for strand,BES_group in self._record.groupby([17]):
            BES_group = BES_group.sort_values(by=[13],ascending=False)
            if BES_group.iloc[0,17] == 'F':
                self._Forward = pd.concat([BES_group,self._Forward])
            elif BES_group.iloc[0,17] == 'R':
                self._Reverse = pd.concat([BES_group,self._Reverse])
                
    def merge(self, BES, edit=6):
        num_rows = BES.shape[0] 
        BES[18] = -1
        if num_rows :
            for first in range(num_rows-1):
                for second in range(first+1,num_rows):
                    if BES.iloc[first,18] != -1 :
                        continue
                    else:
                        if BES.iloc[first,1] == BES.iloc[second,1] and \
                        (abs(BES.iloc[first,10] - BES.iloc[second,10]) < edit or \
                         abs(BES.iloc[first,11] - BES.iloc[second,11]) < edit ):
                            BES.iloc[first,18] = first
                            BES.iloc[second,18] = first
        return BES.drop_duplicates(subset=[18])
    
    def conbinates(self, max_length=200000):
        if self._Forward.shape[0] and self._Reverse.shape[0]:
            FLAG = False
            for F in range(self._Forward.shape[0]):
                for R in range(self._Reverse.shape[0]):
                    start = min(self._Forward.iloc[F, 10], self._Forward.iloc[F, 11], \
                                self._Reverse.iloc[R, 10], self._Reverse.iloc[R, 11])
                    end   = max(self._Forward.iloc[F, 10], self._Forward.iloc[F, 11], \
                                self._Reverse.iloc[R, 10], self._Reverse.iloc[R, 11])
                    if self._Forward.iloc[F, 1] == self._Reverse.iloc[R, 1] and \
                    self._Forward.iloc[F, 14] != self._Reverse.iloc[R, 14]  and \
                    end - start < max_length:
                        self._gff.loc[self._name] = [self._Forward.iat[F,1],'GResource','BES', str(start), str(end), \
                                             '.','.','.',str('ID=' + self._name + ';Name=' + self._name)]
                        self._gff.loc[str(self._name + '-F')] = [self._Forward.iat[F, 1], 'GResource', 'BES_end', \
                                    min(self._Forward.iat[F, 10], self._Forward.iat[F, 11]), \
                                    max(self._Forward.iat[F, 10], self._Forward.iat[F, 11]), \
                                    '.', self._Forward.iat[F, 14], '.', str('Parent=' + self._name)]
                        self._gff.loc[str(self._name + '-R')] = [self._Reverse.iat[R, 1], 'GResource', 'BES_end', \
                                    min(self._Reverse.iat[R, 10], self._Reverse.iat[R, 11]), \
                                    max(self._Reverse.iat[R, 10], self._Reverse.iat[R, 11]), \
                                    '.', self._Reverse.iat[R, 14], '.', str('Parent=' + self._name)]
                    elif self._Forward.iloc[F, 18] != -1:
                        self._gff.loc[self._name] = [self._Forward.iat[F, 1], 'GResource', 'BES_end', \
                                             min(self._Forward.iat[F, 10], self._Forward.iat[F, 11]), \
                                             max(self._Forward.iat[F, 10], self._Forward.iat[F, 11]), \
                                             '.', self._Forward.iat[F, 14], '.', str('ID=' + self._name + ';Name=' + self._name)]
                        FLAG = True
                        break
                    elif self._Reverse.iloc[R, 18] != -1:
                        self._gff.loc[self._name] = [self._Reverse.iat[R, 1], 'GResource', 'BES_end', \
                                             min(self._Reverse.iat[R, 10], self._Reverse.iat[R, 11]), \
                                             max(self._Reverse.iat[R, 10], self._Reverse.iat[R, 11]), \
                                             '.', self._Reverse.iat[R, 14], '.', str('ID=' + self._name + ';Name=' + self._name)]
                        FLAG = True
                        break
                if FLAG:
                    break


        elif self._Forward.shape[0]: # Only forward is recorded
            for F in range(self._Forward.shape[0]):
                self._gff.loc[self._name] = [self._Forward.iat[F, 1], 'GResource', 'BES_end', \
                                             min(self._Forward.iat[F, 10], self._Forward.iat[F, 11]), \
                                             max(self._Forward.iat[F, 10], self._Forward.iat[F, 11]), \
                                             '.', self._Forward.iat[F, 14], '.', str('ID=' + self._name + ';Name=' + self._name)]
        
        elif self._Reverse.shape[0]:   # Only reverse BES is recorded
            for R in range(self._Reverse.shape[0]):
                self._gff.loc[self._name] = [self._Reverse.iat[R, 1], 'GResource', 'BES_end', \
                                             min(self._Reverse.iat[R, 10], self._Reverse.iat[R, 11]), \
                                             max(self._Reverse.iat[R, 10], self._Reverse.iat[R, 11]), \
                                             '.', self._Reverse.iat[R, 14], '.', str('ID=' + self._name + ';Name=' + self._name)]
        




In [2]:
import pandas as pd
import collections


def main(blast):
    #add two field "name" and "end" into 17th and 18th respectively
    for i in range(blast.shape[0]): 
        blast.loc[i, 16] = blast.loc[i, 0].split('_')[0]
        blast.loc[i, 17] = blast.loc[i, 0].split('_')[-1]
    blast[:3]

    # create a new DataFrame with 9 columns of gff3 format
    gff = pd.DataFrame(columns=['chr','source','type','start','end','score','strand','phase','attributes'])

    # group by BAC name
    for BES_name, BES_groups in blast.groupby(16):
        BAC = BAC_Recorder(BES_groups)
        gff = gff.append(BAC._gff)
        
    return gff
if __name__ == '__main__':
    blast = pd.read_table('../workspace/short_and_long.blast',header=None)
    GFF3 = main(blast=blast)
    GFF3.to_csv('PM-B-BES.gff3',index=None,sep='\t')


In [3]:
GFF3

,chr,source,type,start,end,score,strand,phase,attributes
25A01,6,GResource,BES_end,4968299,5008417,.,+,.,ID=25A01;Name=25A01
25A02,18,GResource,BES_end,15834410,15834864,.,-,.,ID=25A02;Name=25A02
25A03,10,GResource,BES,30369637,30469114,.,.,.,ID=25A03;Name=25A03
25A03-F,10,GResource,BES_end,30468872,30469114,.,-,.,Parent=25A03
25A03-R,10,GResource,BES_end,30369637,30369930,.,+,.,Parent=25A03
25A05,2,GResource,BES_end,15041792,15066280,.,+,.,ID=25A05;Name=25A05
25A07,1,GResource,BES,63813657,63903632,.,.,.,ID=25A07;Name=25A07
25A07-F,1,GResource,BES_end,63813657,63903632,.,+,.,Parent=25A07
25A07-R,1,GResource,BES_end,63813657,63903631,.,-,.,Parent=25A07
25A08,8,GResource,BES_end,24289290,24289769,.,-,.,ID=25A08;Name=25A08
